In [14]:
# %pip install FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

loader = TextLoader("textfile.txt")
file = loader.load()

file
######################################
splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=50)

documents = splitter.split_documents(file)

documents
########################################
ollama_embeddings = OllamaEmbeddings(model='nomic-embed-text:latest')



# print(embeddings[0])
# print(embeddings[1])


db = FAISS.from_documents(documents,ollama_embeddings)
db


In [27]:
query = "what is an essay?"
docs = db.similarity_search(query)
docs
text=""
for doc in docs :
    text += str(doc.page_content)

text

"An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding ofKey Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position.Writing Process\nPreparation: Define the topic, research sources, create a thesis, and outline the essay. \nWriting: Draft the introduction, body, and conclusion based on the outline."

In [ ]:
"An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding ofKey Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position.Writing Process\nPreparation: Define the topic, research sources, create a thesis, and outline the essay. \nWriting: Draft the introduction, body, and conclusion based on the outline."

In [ ]:
# retreiver => interface layer

retriever = db.as_retriever()
retriever.invoke(query)

[Document(id='5f1a458a-4f1d-49b1-b71a-2b91e8eba6c0', metadata={'source': 'textfile.txt'}, page_content="An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding of"),
 Document(id='41d76eb9-b33d-48fb-9459-499b7802c4c2', metadata={'source': 'textfile.txt'}, page_content='Key Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:'),
 Document(id='2badf3a8-7681-4695-93ec-89f28574b97d', metadata={'source': 'textfile.txt'}, page_content="settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position."),
 Document(id='f7856b3b-a47a-447c-876c-9165c7e91563', metadata={'source': 'textfile.txt'}, page_content='Writing Process\nPreparation: D

In [17]:
doc_with_score = db.similarity_search_with_score(query)
doc_with_score

[(Document(id='5f1a458a-4f1d-49b1-b71a-2b91e8eba6c0', metadata={'source': 'textfile.txt'}, page_content="An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding of"),
  np.float32(266.14374)),
 (Document(id='41d76eb9-b33d-48fb-9459-499b7802c4c2', metadata={'source': 'textfile.txt'}, page_content='Key Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:'),
  np.float32(347.0409)),
 (Document(id='2badf3a8-7681-4695-93ec-89f28574b97d', metadata={'source': 'textfile.txt'}, page_content="settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position."),
  np.float32(358.1892)),
 (Document(id='f7856b3b-a47a-447c-876c-9165c7e91563', me

In [21]:
#similarity search by vector

vec = ollama_embeddings.embed_query(query)
ssbv = db.similarity_search_by_vector(vec)
ssbv
ssbvscore = db.similarity_search_with_score_by_vector(vec)
ssbvscore

[(Document(id='5f1a458a-4f1d-49b1-b71a-2b91e8eba6c0', metadata={'source': 'textfile.txt'}, page_content="An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding of"),
  np.float32(266.14374)),
 (Document(id='41d76eb9-b33d-48fb-9459-499b7802c4c2', metadata={'source': 'textfile.txt'}, page_content='Key Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:'),
  np.float32(347.0409)),
 (Document(id='2badf3a8-7681-4695-93ec-89f28574b97d', metadata={'source': 'textfile.txt'}, page_content="settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position."),
  np.float32(358.1892)),
 (Document(id='f7856b3b-a47a-447c-876c-9165c7e91563', me

In [ ]:
##save local

db.save_local("faiss_index") #creates a file index , pkl files

In [28]:
new_db = FAISS.load_local("faiss_index",ollama_embeddings,allow_dangerous_deserialization=True)

docs = new_db.similarity_search(query)
docs

[Document(id='5f1a458a-4f1d-49b1-b71a-2b91e8eba6c0', metadata={'source': 'textfile.txt'}, page_content="An essay is a focused piece of writing that develops an idea or argument using evidence, analysis, and interpretation.  It is commonly used in academic settings to assess a student's understanding of"),
 Document(id='41d76eb9-b33d-48fb-9459-499b7802c4c2', metadata={'source': 'textfile.txt'}, page_content='Key Features of an Essay\nPurpose: To present a clear argument or analysis, often in response to a specific question. \nStructure: Most essays follow a three-part structure:'),
 Document(id='2badf3a8-7681-4695-93ec-89f28574b97d', metadata={'source': 'textfile.txt'}, page_content="settings to assess a student's understanding of a topic, with university-level essays typically being argumentative—aiming to persuade the reader of a specific position."),
 Document(id='f7856b3b-a47a-447c-876c-9165c7e91563', metadata={'source': 'textfile.txt'}, page_content='Writing Process\nPreparation: D